## PlantTraits2024 simple, yet effective solution

This notebook was initially inspired by [this great notebook](https://www.kaggle.com/code/markwijkhuizen/planttraits2024-eda-training-pub), also the pickled training data is taken from there. 

The solution pipeline:

1. Clean the data - drop values that are smaller than 0.1% percentile or bigger than 98% percentile of training data;
2. Scale the data - using StandardScaler;
3. Get image embedding from DINOv2 - ([code](https://github.com/facebookresearch/dinov2/tree/main), [paper](https://arxiv.org/abs/2304.07193));
4. Calculate first 1000 2-degree polynomial features - using PolynomialFeatures;
5. Train 6 CatBoost models, one for each target - using embeddings both as numerical features and as embedding features. 

The things that I've also tried but the score was smaller:

* Use XGBoost instead of CatBoost;
* Tune CatBoost - use different losses, training params, use embeddings only as embeddings or only as numerical features;
* Train one CatBoost model instead of 6 models for each target;
* Use different models for embeddings - CLIP, Meta-CLIP, EVA-CLIP, BLIPv2, InternViT and other;
* Blend different models trained on different embeddings;
* Train model like in the [original notebook](https://www.kaggle.com/code/markwijkhuizen/planttraits2024-eda-training-pub) - with image backbone (Swin, ViT, ResNet) and MLP for tabular features;
* Add STD as targets;
* Use different percentiles to clean the data;

In [ ]:
!pip --quiet install fastai xformers

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from catboost import Pool, CatBoostRegressor
from torchvision import transforms

tqdm.pandas()

## Config

In [ ]:
class Config():
    TARGET_COLUMNS = ['X4_mean', 'X11_mean', 'X18_mean',
                      'X26_mean', 'X50_mean', 'X3112_mean']
    # Dataset
    RECOMPUTE_DATAFRAMES_TRAIN = True
    RECOMPUTE_DATAFRAMES_TEST = True
    RECOMPUTE_IMAGE_EMBEDDINGS = False
    N_VAL_SAMPLES0 = 4096
    # Others
    SEED = 42
    DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'


def seed_everything(seed: int):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


CONFIG = Config()
seed_everything(CONFIG.SEED)
CONFIG.DEVICE

## Load & split DataFrames

In [ ]:
# load pickled dataframes from a public dataset; split to train-val
if CONFIG.RECOMPUTE_DATAFRAMES_TRAIN:
    train0 = pd.read_csv('/kaggle/input/cs-480-2024-spring/data/train.csv')
    train0['file_path'] = train0['id'].apply(
        lambda s: f'/kaggle/input/cs-480-2024-spring/data/train_images/{s}.jpeg')
else:
    train0 = pd.read_pickle(
        '/kaggle/input/planttraits2024-eda-training-pub-dataset/train.pkl')

if CONFIG.RECOMPUTE_DATAFRAMES_TEST:
    test = pd.read_csv('/kaggle/input/cs-480-2024-spring/data/test.csv')
    test['file_path'] = test['id'].apply(
        lambda s: f'/kaggle/input/cs-480-2024-spring/data/test_images/{s}.jpeg')
else:
    test = pd.read_pickle(
        '/kaggle/input/planttraits2024-eda-training-pub-dataset/test.pkl')
CONFIG.FEATURE_COLUMNS = test.columns.values[1:-2]

train, val = train_test_split(
    train0, test_size=CONFIG.N_VAL_SAMPLES0, shuffle=True, random_state=CONFIG.SEED)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

## Filter outliers

In [ ]:
def get_mask(df, labels_describe_df):
    lower = []
    higher = []
    mask = np.empty(shape=df[CONFIG.TARGET_COLUMNS].shape, dtype=bool)
    for idx, t in enumerate(CONFIG.TARGET_COLUMNS):
        labels = df[t].values
        v_min, v_max = labels_describe_df.loc[t]['0.1%'], labels_describe_df.loc[t]['98%']
        mask[:, idx] = ((labels > v_min) & (labels < v_max))
    return mask.min(axis=1)


labels_describe_df = train[CONFIG.TARGET_COLUMNS].describe(
    percentiles=[0.001, 0.98]).round(3).T
# Masks
mask_train = get_mask(train, labels_describe_df)
mask_val = get_mask(val, labels_describe_df)
# Masked DataFrames
train_mask = train[mask_train].reset_index(drop=True)
val_mask = val[mask_val].reset_index(drop=True)

for m, subset, full in zip([train_mask, val_mask], ['train', 'val'], [train, val]):
    print(f'===== {subset} shape: {m.shape} =====')
    n_masked = len(full) - len(m)
    perc_masked = (n_masked / len(full)) * 100
    print(f'{subset} \t| # Masked Samples: {n_masked}')
    print(f'{subset} \t| % Masked Samples: {perc_masked:.3f}%')

## Process features

In [ ]:
# Standard Scaler for Features
FEATURE_SCALER = StandardScaler()
# Fit and transform on training features
train_features_mask = FEATURE_SCALER.fit_transform(
    train_mask[CONFIG.FEATURE_COLUMNS].values.astype(np.float32))
# Transform val/test features using scaler fitted on train data
val_features_mask = FEATURE_SCALER.transform(
    val_mask[CONFIG.FEATURE_COLUMNS].values.astype(np.float32))
test_features = FEATURE_SCALER.transform(
    test[CONFIG.FEATURE_COLUMNS].values.astype(np.float32))

y_train_mask = train_mask[CONFIG.TARGET_COLUMNS].values
y_val_mask = val_mask[CONFIG.TARGET_COLUMNS].values

## Get DINO image embeddings

In [ ]:
def get_image_embeddings_dino(model, preprocess, batch_size, df):
    image_embeddings = []
    for i in tqdm(range(0, len(df), batch_size)):
        paths = df['file_path'][i:i + batch_size]
        image_tensor = torch.stack(
            [preprocess(Image.open(path)) for path in paths]).to(CONFIG.DEVICE)
        with torch.no_grad():
            curr_image_embeddings = model(image_tensor)
        image_embeddings.extend(curr_image_embeddings.cpu().numpy())
    return image_embeddings

In [ ]:
if CONFIG.RECOMPUTE_IMAGE_EMBEDDINGS:
    model = torch.hub.load('facebookresearch/dinov2',
                           'dinov2_vitg14_reg').to(CONFIG.DEVICE)
    model.eval()
    # the preprocessing differs from the original code, originally it was resize + crop
    # but we lose info while cropping, so here we use only resize to 224
    preprocess = transforms.Compose([
        transforms.Resize(224, interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize((0.4464, 0.4505, 0.3360),
                             (0.1834, 0.1794, 0.1779)),
    ])

    batch_size = 64
    suffix = 'image_embs_dinov2_vitg14_reg'
    train_image_embeddings = get_image_embeddings_dino(
        model, preprocess, batch_size, train_mask)
    np.save(f'train_{suffix}', np.array(train_image_embeddings))
    val_image_embeddings = get_image_embeddings_dino(
        model, preprocess, batch_size, val_mask)
    np.save(f'val_{suffix}', np.array(val_image_embeddings))
    test_image_embeddings = get_image_embeddings_dino(
        model, preprocess, batch_size, test)
    np.save(f'test_{suffix}', np.array(test_image_embeddings))
else:
    suffix = 'image_embs_dinov2_vitg14_reg'
    train_image_embeddings = np.load(
        f'/kaggle/input/cs480-sping-2024-image-embeddings/train_{suffix}.npy')
    val_image_embeddings = np.load(
        f'/kaggle/input/cs480-sping-2024-image-embeddings/val_{suffix}.npy')
    test_image_embeddings = np.load(
        f'/kaggle/input/cs480-sping-2024-image-embeddings/test_{suffix}.npy')
    print(f'Embeddings {suffix} loaded from dataset.')

## Get final features DataFrames

In [ ]:
# we can potentially use all the polynomial features but it would take an etenriny to train the models
first_n_poly_feats = 1000
train_features_mask_all = np.concatenate(
    (PolynomialFeatures(2).fit_transform(train_features_mask)
     [:, :first_n_poly_feats], train_image_embeddings), axis=1
)
val_features_mask_all = np.concatenate(
    (PolynomialFeatures(2).fit_transform(val_features_mask)
     [:, :first_n_poly_feats], val_image_embeddings), axis=1
)
test_features_all = np.concatenate(
    (PolynomialFeatures(2).fit_transform(test_features)[
     :, :first_n_poly_feats], test_image_embeddings), axis=1
)

In [ ]:
train_features_mask_df = pd.DataFrame(train_features_mask_all)
train_features_mask_df['emb'] = list(train_image_embeddings)

val_features_mask_df = pd.DataFrame(val_features_mask_all)
val_features_mask_df['emb'] = list(val_image_embeddings)

test_features_mask_df = pd.DataFrame(test_features_all)
test_features_mask_df['emb'] = list(test_image_embeddings)

## Train CatBoost

In [ ]:
models = {}
scores = {}
for i, col in tqdm(enumerate(CONFIG.TARGET_COLUMNS), total=len(CONFIG.TARGET_COLUMNS)):
    y_curr = y_train_mask[:, i]
    y_curr_val = y_val_mask[:, i]
    train_pool = Pool(train_features_mask_df, y_curr,
                      embedding_features=['emb'])
    val_pool = Pool(val_features_mask_df, y_curr_val,
                    embedding_features=['emb'])

    # tried to tune these parameters but without real success
    model = CatBoostRegressor(
        iterations=1900,
        learning_rate=0.075,
        loss_function='RMSE',
        verbose=0,
        random_state=CONFIG.SEED,
        task_type="GPU",
        devices='0'
    )

    model.fit(train_pool)
    models[col] = model

    y_curr_val_pred = model.predict(val_pool)

    r2_col = r2_score(y_curr_val, y_curr_val_pred)
    scores[col] = r2_col
    print(f'Target: {col}, R2: {r2_col:.3f}')
# this val score somewhat correlates with submission score bit I didn't really bother
print(f'Mean R2: {np.mean(list(scores.values())):.3f}')

## Submission

In [ ]:
submission = pd.DataFrame({'id': test['id']})
submission[CONFIG.TARGET_COLUMNS] = 0
submission.columns = submission.columns.str.replace('_mean', '')
for i, col in enumerate(CONFIG.TARGET_COLUMNS):
    test_pool = Pool(test_features_mask_df, embedding_features=['emb'])
    col_pred = models[col].predict(test_pool)
    submission[col.replace('_mean', '')] = col_pred

submission.to_csv('submission_epoch1900_lr0.075_1.csv', index=False)
submission.head()